In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install -U pymoo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 249 kB 41.5 MB/s 
     |████████████████████████████████| 207 kB 50.4 MB/s 
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210096 sha256=e52229cd362e4e000e6320a76e8d1d39bd5081a4b26df0903a29f67c73ddb2d4
  Stored in directory: /root/.cache/pip/wheels/71/fc/c8/142ef03b79c02b618fe80b9f2d01c25bd55f60b0c58eab3f0e
Successfully built grapheme


In [ ]:
import colorsys
import random
import pickle
import numpy as np
import os
import xml.etree.ElementTree as ET
from datetime import datetime
from PIL import Image
import csv
from pathlib import Path
import re

from pymoo.core.sampling import Sampling
from pymoo.core.problem import ElementwiseProblem
from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.core.crossover import Crossover
from pymoo.core.mutation import Mutation
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from multiprocessing.pool import ThreadPool
from pymoo.core.problem import StarmapParallelization
from pymoo.visualization.scatter import Scatter

# **Return the palette of colors in rgb**

In [ ]:
# convert hsv to rgb
def getRGBfromHsv(hsl):
    # normalized the h,s,v for colorsys
    h = hsl[0] / 360
    s = hsl[1]
    v = hsl[2]
    return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h, s, v))

In [ ]:
# random colors from HSB/HSV
def getEquidistantHarmony(length):
    list_eh = []

    # random start_hue but with fixed saturation and brightness
    start_h = random.randint(0, 360)
    h = start_h
    value_add = int(360 / length)
    s = random.uniform(0, 1)
    l = random.uniform(0.2, 0.9)
    # iterate for the length passed in input
    for _ in range(0, length):
        # same saturation and brightnes, but modulating the tone (hue)
        hsl = (h, s, l)
        list_eh.append(hsl)
        # module for stay between 0 and 360
        h = (h + value_add) % 360
    return list_eh

In [ ]:
def getEHContrast(equidistant_harmony):
    list_eh_contrast = []
    # iterate for the length passed in input
    for i in range(0, len(equidistant_harmony)):
        # random saturation and brightnes, but with the same tone of list (hue)
        h = equidistant_harmony[i][0]
        s = random.uniform(0, 1)
        v = random.uniform(0.2, 0.9)
        hsv = (h, s, v)
        list_eh_contrast.append(getRGBfromHsv(hsv))
    return list_eh_contrast

In [ ]:
def getMonocromaticPalette(length):
    h = random.randint(0, 360)
    list_monochromatic = []
    # iterate for the length passed in input
    for i in range(0, length):
        # random saturation and brightnes, but with the same color (hue)
        s = random.uniform(0, 1)
        v = random.uniform(0.2, 0.9)
        hsv = (h, s, v)
        list_monochromatic.append(getRGBfromHsv(hsv))
    return list_monochromatic

In [ ]:
# return a palette list of 512 colors in rgb
def getNewPalette(list_color):
    color_palette = []
    # (i) m colors from the original BOCP
    color_palette.extend(list_color)
    # m length
    m = len(list_color)
    # (ii) white
    white = (255, 255, 255)
    color_palette.append(white)
    # (iii) black
    black = (0, 0, 0)
    color_palette.append(black)
    # formula of paper for calculate numbers of colors
    num = (512 - m - 2) / 3
    # check if there are decimal values from the module
    if num % 1 == 0:
        # delete decimal values
        num = int(num)
        # (iv) a palette in equidistant color harmony with (512-m-2)/3 colors
        equidistant_harmony = getEquidistantHarmony(num)
    else:
        # delete decimal values
        num = int(num)
        # I add a color otherwise we don't get to 512
        equidistant_harmony = getEquidistantHarmony(num)
    for color in equidistant_harmony:
        color_palette.append(getRGBfromHsv(color))

    # (v) a monochromatic palette of (512-m-2)/3 colors with difference in saturation and brightness
    monochromatic = getMonocromaticPalette(num + 1)
    color_palette.extend(monochromatic)

    # (vi) a equidistant harmony palette with random saturation and brightness for each of the colors in the harmony
    equidistant_harmony_contrast = getEHContrast(equidistant_harmony)
    color_palette.extend(equidistant_harmony_contrast)

    return color_palette

# **Return the first fitness function values, we need minimize it**

In [ ]:
def PowerPixel(color):
    return (color[0] * 76.23 + 0.23) + (color[1] * 89.97 + 2.34) + (color[2] * 148.40 + 7.30)

# ritorna l'indice dell'array di bocp per usarlo con S
def CounterIndex(BOCP, h):
    list_keys = list(BOCP.keys())
    n_values = [0] * len(list_keys)
    for index, key in enumerate(list_keys):
        for pixel in BOCP.get(key):
            if pixel[0] == h:
                n_values[index] += 1
    return n_values

# estimated power consumptions of all the pixels in all the screens in Joule
def ECF(bocp, solution, interface):
    total_consumption = 0
    lista_by_colori = []
    # iterate all colors
    for h in range(len(interface)):
        n_values = CounterIndex(bocp, h)
        for i, s in enumerate(solution):
            total_consumption += (PowerPixel(s) * n_values[i]) * interface[h]
    return total_consumption

# **Return the second fitness function values, we need maximize it**

In [ ]:
# function to take all GUI's
def getSnapshot(path):
    data = []
    for filename in sorted(os.listdir(path)):
        if filename.endswith("uix"):
            tree = ET.parse(path + "/" + filename)
            # getting the parent tag of the xml document
            root = tree.getroot()
            data.append(root)
    return data


# Con function calculate the contrast between two colors a and b (in terms of brightness)
def Con(color_a, color_b):
    return abs(((299 * color_a[0] + 587 * color_a[1] + 114 * color_a[2]) / 1000) - (
            (299 * color_b[0] + 587 * color_b[1] + 114 * color_b[2]) / 1000))

# check if the elements are equal
def Comp(root, comp):
    for elem in root.iter():
        if elem.items() == comp.items():
            return elem

# ritorna l'insieme di tutte le componenti adiacenti, padre diretto e figli diretti
def getAdiacentComponents(root, comp):
    parent_map = {c: p for p in root.iter() for c in p}
    gen = parent_map.get(Comp(root, comp))
    if gen == None:
        return None
    adiac = []
    for elem in root.iter():
        if elem == gen:
            adiac.append(elem)
            adiac.extend(list(comp))
            return adiac

def BCIndexWithH(BOCC, c, h):
    list_keys = list(BOCC.keys())
    for index, key in enumerate(list_keys):
        for comp in BOCC.get(key):
            if comp[0] == h and comp[1].items() == c.items():
                return index
    return None

def TCon(solution, bocc, root, h, CnTh):
    total_con = np.array([0, 0])
    flag = False
    # iterate all components in snapshot
    for elem in root.iter():
        if (len(elem.items()) == 17 and elem.items()[3][1].find("ImageView") == -1 and elem.items()[3][1].find("ImageButton") == -1) or (
                len(elem.items()) == 18 and elem.items()[4][1].find("ImageView") == -1 and elem.items()[4][1].find("ImageButton") == -1):
            flag = True
            list_comp = getAdiacentComponents(root, elem)
        if flag:
            flag = False
            if list_comp is not None:
                first_comp = BCIndexWithH(bocc, elem, h)
                if first_comp is not None:
                    solution_1 = solution[first_comp]
                    for adiac in list_comp:
                        if (len(adiac.items()) == 17 and adiac.items()[3][1].find("ImageView") == -1 and adiac.items()[3][1].find("ImageButton") == -1) or (
                                len(adiac.items()) == 18 and adiac.items()[4][1].find("ImageView") == -1 and adiac.items()[4][1].find("ImageButton") == -1):
                            second_comp = BCIndexWithH(bocc, adiac, h)
                            if second_comp is not None and (adiac.items()!=elem.items()):
                                solution_2 = solution[second_comp]
                                contrasto = Con(solution_1, solution_2)
                                total_con[0] += contrasto
                                if contrasto < CnTh and first_comp!=second_comp:
                                    total_con[1] += 1
                                    
    return total_con

# calculate second fitness function
def CF(Solution, bocc, snapshot, CnTh):
    # consumo
    consumption = [0, 0]
    # importiamo il file
    path = "Snapshot/" + snapshot
    # lista contenente tutte le root e passarli uno alla volta col ciclo sotto
    screen = getSnapshot(path)
    for h, interface in enumerate(screen):
        t_con = TCon(Solution, bocc, interface, h, CnTh)
        consumption[0] += t_con[0]
        consumption[1] += t_con[1]
    return consumption

# **Return the third fitness function values, we need minimize it**

In [ ]:
# Euclidean Distance of rgb values
def EuclideanDistance(color1, color2):
    a = np.array((color1[0], color1[1], color1[2]))
    b = np.array((color2[0], color2[1], color2[2]))

    dist = np.sqrt(np.sum(np.square(a - b)))
    return dist

# Or function, given a color returns the closest color in the original palette
def OrStar(color, palette):
    closest = palette[0]
    min_dist = EuclideanDistance(color, closest)
    for p_color in palette:
        if min_dist > EuclideanDistance(color, p_color):
            closest = p_color
            min_dist = EuclideanDistance(color, p_color)
    return closest

# penalty factor
def alpha(color, bocp):
    if color != OrStar(color, bocp):
        return 1
    else:
        return 2

def DF(Solution, initial_colors):
    tot_value = 0
    for sol in Solution:
        the_alpha = alpha(sol, initial_colors)
        the_euclid = EuclideanDistance(sol, OrStar(sol, initial_colors))
        tot_value += the_alpha * the_euclid
    return tot_value

# **Problem**

In [ ]:
class MyProblem(ElementwiseProblem):
    def __init__(self, bocp, bocc, name_snap, interface, **kwargs):
        super().__init__(n_var=1, n_obj=3,n_ieq_constr=1, **kwargs)
        # bocp and bocc hashmap
        self.bocp = bocp
        self.bocc = bocc
        # path of snapshot of apps
        self.name_snap = name_snap
        # list initial colors and his length
        self.initial_colors = list(bocp.keys())
        # palette
        self.palette = getNewPalette(self.initial_colors)
        self.interface = interface
        self.n_colors = len(self.initial_colors)
        self.CnTh = 4

    def _evaluate(self, x, out, *args, **kwargs):
        # fitness function and constraint value
        f1 = ECF(self.bocp, x[0], self.interface)
        f2_g1 = CF(x[0], self.bocc, self.name_snap, self.CnTh)
        f2, g1 = f2_g1[0], f2_g1[1]
        f3 = DF(x[0], self.initial_colors)
        out["F"] = np.array([f1, -f2, f3], dtype=float)
        out["G"] = [g1]


# **Sampling**

In [ ]:
class MySampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        n_colors = problem.n_colors
        palette = problem.palette
        X = np.full((n_samples, 1), None, dtype=object)
        for i in range(n_samples):
            X[i, 0] = tuple([random.choice(palette) for _ in range(n_colors)])
        return X

# **Selection**

In [ ]:
# choose the best individual from pool/tournament with probability p
# choose the second best individual with probability p*(1-p)
# the third with p*((1-p)^2)
# and so on
# simple binary tournament for a single-objective algorithm
def binary_tournament(pop, P, _, **kwargs):
    # The P input defines the tournaments and competitors
    n_tournaments, n_competitors = P.shape

    if n_competitors != 2:
        raise Exception("Only pressure=2 allowed for binary tournament!")
    # the result this function returns
    S = np.full(n_tournaments, None, dtype=object)
    # now do all the tournaments
    for i in range(n_tournaments):
        a, b = P[i]
        # if have same cv
        if pop[a].CV == 0 and pop[b].CV == 0:
            # take the first
            if pop[a].F < pop[b].F:
                S[i] = a
            # otherwise take the other individual
            else:
                # take the second
                S[i] = b
        elif pop[a].CV == 0:
            S[i] = a
        elif pop[b].CV == 0:
            S[i] = b
        elif pop[a].CV < pop[b].CV:
            S[i] = a
        elif pop[a].CV > pop[b].CV:
            S[i] = b
        else:
            if pop[a].F < pop[b].F:
                S[i] = a
            # otherwise take the other individual
            else:
                S[i] = b
    return S

# **Crossover**

In [ ]:
class MyCrossover(Crossover):
    def __init__(self, p_cross):
        # define the crossover: number of parents and number of offsprings
        super().__init__(2, 2)
        self.p_cross = p_cross

    def _do(self, problem, X, **kwargs):

        # The input of has the following shape (n_parents, n_matings, n_var)
        _, n_matings, n_var = X.shape

        # The output owith the shape (n_offsprings, n_matings, n_var)
        # Because there the number of parents and offsprings are equal it keeps the shape of X
        Y = np.full_like(X, None, dtype=object)

        # for each mating provided
        for k in range(n_matings):

            # get the first and the second parent
            a, b = X[0, k, 0], X[1, k, 0]

            # prepare the offsprings
            value = (1, 2, 3)
            off_a = list((value,) * problem.n_colors)
            off_b = list((value,) * problem.n_colors)

            if np.random.random() < self.p_cross:
                position = random.randint(0, problem.n_colors)
                for i in range(problem.n_colors):
                    if i < position:
                        off_a[i] = a[i]
                        off_b[i] = b[i]
                    else:
                        off_a[i] = b[i]
                        off_b[i] = a[i]
            else:
                # prepare the offsprings
                off_a = a
                off_b = b

            # join the character list and set the output
            Y[0, k, 0], Y[1, k, 0] = tuple(off_a), tuple(off_b)

        return Y

# **Mutation**

In [ ]:
class MyMutation(Mutation):
    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):
        p_mut = 1 / len(X)
        # for each individual
        for i in range(len(X)):
            mut = list(X[i, 0])
            for j in range(len(mut)):
                if np.random.random() < p_mut:
                    mut[j] = random.choice(problem.palette)
            X[i, 0] = mut

        return X

# **Duplicates**

In [ ]:
class MyDuplicateElimination(ElementwiseDuplicateElimination):
    def is_equal(self, a, b):
        return a.X[0] == b.X[0]

# **Main**

In [ ]:
# function to take all dir
def getdir(path):
    return sorted(os.listdir(path))

# function to take all GUI's
def getScreenshot(path):
    data = []
    for filename in sorted(os.listdir(path)):
        if filename.endswith("png"):
            data.append(path + "/" + filename)
    return data

In [ ]:
dir_path = "Snapshot"

header = ['id', 'app_name', 'algorithm', 'n_thread', 'n_interface', 'pop_size', 'time_execution', 'n_evals',
          'hist_cv', 'hist_cv_avg']

# create csv for collect data
with open('Risultati/Data-NSGAII.csv', 'w') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    f.close()
    # iterate all dir
for index, dir in enumerate(getdir(dir_path)):
  with open('Risultati/Data-NSGAII.csv', 'a') as f1:
    writer = csv.writer(f1)
    # create data
    data = []
    data.append(index)
    # datetime object containing current date and time
    now = datetime.now()
    # collect app name
    data.append(dir)
    data.append("NSGAII")
    # load pickle data
    bocp_path = "Dati/" + dir + "_BOCC.pickle"
    with open(bocp_path, 'rb') as handle:
      BOCC = pickle.load(handle)
    bocc_path = "Dati/" + dir + "_BOCP.pickle"
    with open(bocc_path, 'rb') as handle:
      BOCP = pickle.load(handle)
    hash_bocc = {}
    hash_comp = {}
    # generate new interface
    old_colors = list(BOCP.keys())

    #delete duplicates
    for key in old_colors:
      for component in BOCC.get(key):
        comp = (component[0], component[1])
        if hash_comp.get(comp) is None:
          hash_comp.setdefault(comp, [])
          components = (component[0], component[1])
          hash_bocc.setdefault(key, []).append(components)

    # initialize the thread pool and create the runner
    n_thread = 10
    data.append(n_thread)
    pool = ThreadPool(n_thread)
    runner = StarmapParallelization(pool.starmap)
    # collect number of interface
    path = "Snapshot/" + dir
    list_images = getScreenshot(path)
    interface = [1] * len(list_images)
    data.append(len(interface))

    # declare problem
    problem = MyProblem(BOCP, hash_bocc, dir, interface, elementwise_runner=runner)
    pf = problem.pareto_front()

    # define tournament
    selection = TournamentSelection(pressure=2, func_comp=binary_tournament)
    p_cross = 0.9
    # collect n_eval
    n_eval = 5000
    termination = get_termination("n_eval", n_eval)
    # collect pop size
    pop_size = 50
    data.append(pop_size)
    # define algorithm and start
    algorithm = NSGA2(pop_size=pop_size,
                      sampling=MySampling(),
                      crossover=MyCrossover(p_cross),
                      mutation=MyMutation(),
                      eliminate_duplicates=MyDuplicateElimination())

    res = minimize(problem,
                  algorithm,
                  termination,
                  pf=pf,
                  seed=1,
                  save_history=True,
                  verbose=True)
    # collect time execution
    end = datetime.now()
    data.append(end - now)
    # close thread
    pool.close()
    # collect history data of execution
    hist = res.history
    n_evals = []  # corresponding number of function evaluations\
    hist_F = []  # the objective space values in each generation
    hist_cv = []  # constraint violation in each generation
    hist_cv_avg = []  # average constraint violation in the whole population

    for algo in hist:
      # store the number of function evaluations
      n_evals.append(algo.evaluator.n_eval)
      # retrieve the optimum from the algorithm
      opt = algo.opt
      # store the least contraint violation and the average in each population
      hist_cv.append(opt.get("CV").min())
      hist_cv_avg.append(algo.pop.get("CV").mean())
      # filter out only the feasible and append and objective space values
      feas = np.where(opt.get("feasible"))[0]
      hist_F.append(opt.get("F")[feas])

    data.append(n_evals)
    data.append(hist_cv)
    data.append(hist_cv_avg)
    writer.writerow(data)
    f1.close()
    # write the data for individual app
    header_app = ['id', 'app_name', 'solution', 'fitness function']
    with open('Risultati/Data-' + dir + '-NSGAII.csv', 'w') as f2:
      writer = csv.writer(f2)
      # write the header
      writer.writerow(header_app)
      for id, solution in enumerate(res.X):
        data_app = []
        data_app.append(id)
        data_app.append(dir)
        data_app.append(solution[0])
        ff = (res.F[id][0], res.F[id][1], res.F[id][2])
        data_app.append(ff)
        writer.writerow(data_app)
        f2.close()